In [1]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction\experiments\model_training
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv(r'artifacts/transformed_data/train_transformed.csv')
test_df = pd.read_csv(r'artifacts/transformed_data/test_transformed.csv')

## Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    train_data : Path
    test_data : Path
    model_dir : Path
    model_ojb : str
    n_estimators : int
    oob_score : bool

## configuration

In [8]:
from churnPredictor.constants import *
from churnPredictor.utils import *

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])
        
    
    def get_modelTrainer_config(self):
        config = self.config.data_transformation
        params = self.params.RandomForest

        create_dirs([config.model_dir])
        

        return ModelTrainerConfig(
            train_data=config.train_data,
            test_data=config.test_data,
            model_dir=config.model_dir,
            model_ojb=config.model_ojb,
            n_estimators=params.n_estimators,
            oob_score=params.oob_score)
